<a href="https://colab.research.google.com/github/Sai-sakunthala/Assignment-3/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch wandb pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink-

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
import wandb
import editdistance
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
LANG = 'te'
data_path = f'/content/drive/MyDrive/dakshina_dataset_v1.0/{LANG}/lexicons/'

def read_data(filepath, max_len=40):
    pairs = []
    with open(filepath, encoding='utf8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) < 2:
                continue
            source, target = parts[0], parts[1]
            if len(source) <= max_len and len(target) <= max_len:
                pairs.append((source, target))
    return pairs

def make_vocab(sequences):
    vocab = {'<pad>':0, '<sos>':1, '<eos>':2}
    idx = 3
    for seq in sequences:
        for ch in seq:
            if ch not in vocab:
                vocab[ch] = idx
                idx += 1
    idx2char = {i:c for c,i in vocab.items()}
    return vocab, idx2char

def encode_word(word, vocab):
    return [vocab['<sos>']] + [vocab[ch] for ch in word] + [vocab['<eos>']]

def pad_seq(seq, max_len, pad_idx=0):
    return seq + [pad_idx] * (max_len - len(seq))

class TransliterationDataset(Dataset):
    def __init__(self, pairs, source_vocab, target_vocab):
        self.source_pad = source_vocab['<pad>']
        self.target_pad = target_vocab['<pad>']
        self.data = []
        for source, target in pairs:
            source_t = encode_word(source, source_vocab)
            target_t = encode_word(target, target_vocab)
            self.data.append((source_t, target_t))
        self.source_max = max(len(x[0]) for x in self.data)
        self.target_max = max(len(x[1]) for x in self.data)

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        source, target = self.data[idx]
        source = pad_seq(source, self.source_max, self.source_pad)
        target = pad_seq(target, self.target_max, self.target_pad)
        return torch.tensor(source), torch.tensor(target)

class translit_Encoder(nn.Module):
    def __init__(self, input_dimensions, emb_dimensions, hid_dimensions, num_layers, dropout, cell='lstm'):
        super().__init__()
        self.embedding = nn.Embedding(input_dimensions, emb_dimensions)
        rnn_cls = {'rnn': nn.RNN, 'gru': nn.GRU, 'lstm': nn.LSTM}[cell.lower()]
        self.rnn = rnn_cls(emb_dimensions, hid_dimensions, num_layers, dropout=dropout if num_layers > 1 else 0, batch_first=True)
        self.cell = cell.lower()

    def forward(self, source):
        embedded = self.embedding(source)
        if self.cell == 'lstm':
            outputs, (hidden, cell) = self.rnn(embedded)
            return hidden, cell
        else:
            outputs, hidden = self.rnn(embedded)
            return hidden, None

class translit_Decoder(nn.Module):
    def __init__(self, output_dimensions, emb_dimensions, hid_dimensions, num_layers, dropout, cell='lstm'):
        super().__init__()
        self.embedding = nn.Embedding(output_dimensions, emb_dimensions)
        rnn_cls = {'rnn': nn.RNN, 'gru': nn.GRU, 'lstm': nn.LSTM}[cell.lower()]
        self.rnn = rnn_cls(emb_dimensions, hid_dimensions, num_layers, dropout=dropout if num_layers > 1 else 0, batch_first=True)
        self.fc_out = nn.Linear(hid_dimensions, output_dimensions)
        self.cell = cell.lower()

    def forward(self, input, hidden, cell=None):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        if self.cell == 'lstm':
            output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        else:
            output, hidden = self.rnn(embedded, hidden)
            cell = None
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

class translit_Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        output_dimensions = self.decoder.fc_out.out_features

        outputs = torch.zeros(batch_size, target_len, output_dimensions).to(self.device)
        hidden, cell = self.encoder(source)
        input = target[:, 0]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = target[:, t] if teacher_force else top1
        return outputs

def strip_after_eos(seq, eos_idx):
    if eos_idx in seq:
        return seq[:seq.index(eos_idx) + 1]
    return seq

def calculate_word_accuracy(preds, targets, pad_idx=0, eos_idx=None):
    correct = 0
    total = 0
    for pred, target in zip(preds, targets):
        if eos_idx is not None:
            pred = strip_after_eos(pred, eos_idx)
            target = strip_after_eos(target, eos_idx)
        pred = [p for p in pred if p != pad_idx]
        target = [t for t in target if t != pad_idx]
        if pred == target:
            correct += 1
        total += 1
    return correct / total if total > 0 else 0

def calculate_accuracy(preds, targets, pad_idx=0):
    total = 0
    correct = 0
    for p, t in zip(preds, targets):
        for pi, ti in zip(p, t):
            if ti == pad_idx:
                continue
            if pi == ti:
                correct += 1
            total += 1
    return correct / total if total > 0 else 0

def calculate_cer(preds, targets, pad_idx=0):
    cer, total = 0, 0
    for pred, target in zip(preds, targets):
        # Remove PAD tokens for evaluation.
        pred = [p for p in pred if p != pad_idx]
        target = [t for t in target if t != pad_idx]
        cer += editdistance.eval(pred, target)
        total += len(target)
    return cer / total if total > 0 else 0

def run(config=None):
    with wandb.init(config=config):
        cfg = wandb.config
        cfg.hidden_dim = 2 * cfg.embed_dim if cfg.hidden_dim_config == 'double' else cfg.embed_dim
        sweep_name = f"{cfg.cell_type}_{cfg.embed_dim}e_{cfg.hidden_dim_config}h_{cfg.layers}l_" \
             f"{int(cfg.dropout*100)}d_{int(cfg.teacher_forcing*10)}tf_" \
             f"{str(cfg.lr).replace('.', '')}lr"

        wandb.run.name = sweep_name

        max_len = 30

        # Load data (ensure paths match your dataset—full dataset naming assumed)
        train_pairs = read_data(data_path + f"{LANG}.translit.sampled.train.tsv", max_len=max_len)
        val_pairs   = read_data(data_path + f"{LANG}.translit.sampled.dev.tsv",   max_len=max_len)
        test_pairs  = read_data(data_path + f"{LANG}.translit.sampled.test.tsv",  max_len=max_len)

        source_vocab, _ = make_vocab([x[0] for x in train_pairs])
        target_vocab, _ = make_vocab([x[1] for x in train_pairs])

        # (Add assertions to ensure special tokens are consistent)
        assert source_vocab['<pad>'] == 0 and target_vocab['<pad>'] == 0, "Pad token must be index 0 in both vocabs."

        train_translit = TransliterationDataset(train_pairs, source_vocab, target_vocab)
        val_translit   = TransliterationDataset(val_pairs,   source_vocab, target_vocab)
        test_translit  = TransliterationDataset(test_pairs,  source_vocab, target_vocab)

        # Use drop_last=True to ensure consistent batch sizes.
        train_drop_last = DataLoader(train_translit, batch_size=cfg.batch_size, shuffle=True, drop_last=True)
        val_drop_last   = DataLoader(val_translit,   batch_size=cfg.batch_size, drop_last=True)
        test_drop_last  = DataLoader(test_translit,  batch_size=cfg.batch_size, drop_last=True)

        encoder = translit_Encoder(len(source_vocab), cfg.embed_dim, cfg.hidden_dim, cfg.layers, cfg.dropout, cfg.cell_type).to(device)
        decoder = translit_Decoder(len(target_vocab), cfg.embed_dim, cfg.hidden_dim, cfg.layers, cfg.dropout, cfg.cell_type).to(device)
        model = translit_Seq2Seq(encoder, decoder, device).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr)
        criterion = nn.CrossEntropyLoss(ignore_index=target_vocab['<pad>'])
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

        best_val_loss = float('inf')
        patience = 10
        wait = 0

        for epoch in range(cfg.epochs):
            model.train()
            total_loss = 0
            total_acc  = 0
            total_char_acc = 0
            for source, target in train_drop_last:
                source, target = source.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(source, target, cfg.teacher_forcing)
                out_dimensions = output.shape[-1]
                loss = criterion(output[:, 1:].reshape(-1, out_dimensions), target[:, 1:].reshape(-1))
                eos_idx = target_vocab['<eos>']

                raw_preds = output.argmax(2)[:, 1:].tolist()
                raw_targets = target[:, 1:].tolist()
                preds = [strip_after_eos(p, eos_idx) for p in raw_preds]
                targets = [strip_after_eos(t, eos_idx) for t in raw_targets]
                acc = calculate_word_accuracy(preds, targets, pad_idx=target_vocab['<pad>'])
                char_acc = calculate_accuracy(preds, targets, pad_idx=target_vocab['<pad>'])
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                total_loss += loss.item()
                total_acc  += acc
                total_char_acc += char_acc

            avg_train_loss = total_loss / len(train_drop_last)
            avg_train_acc  = total_acc / len(train_drop_last)
            avg_train_char_acc = total_char_acc / len(train_drop_last)

            model.eval()
            val_loss = 0
            val_acc  = 0
            val_cer  = 0
            total_char_acc = 0
            with torch.no_grad():
                for source, target in val_drop_last:
                    source, target = source.to(device), target.to(device)
                    output = model(source, target, teacher_forcing_ratio=0)
                    out_dimensions = output.shape[-1]
                    loss = criterion(output[:, 1:].reshape(-1, out_dimensions), target[:, 1:].reshape(-1))
                    eos_idx = target_vocab['<eos>']

                    raw_preds = output.argmax(2)[:, 1:].tolist()
                    raw_targets = target[:, 1:].tolist()
                    preds = [strip_after_eos(p, eos_idx) for p in raw_preds]
                    targets = [strip_after_eos(t, eos_idx) for t in raw_targets]
                    acc = calculate_word_accuracy(preds, targets, pad_idx=target_vocab['<pad>'])
                    char_acc = calculate_accuracy(preds, targets, pad_idx=target_vocab['<pad>'])
                    val_loss += loss.item()
                    val_acc  += acc
                    preds = output.argmax(2).tolist()
                    targets = target.tolist()
                    val_cer += calculate_cer(preds, targets, pad_idx=target_vocab['<pad>'])
                    total_char_acc += char_acc

            avg_val_loss = val_loss / len(val_drop_last)
            avg_val_acc  = val_acc / len(val_drop_last)
            avg_val_cer  = val_cer / len(val_drop_last)
            avg_val_char_acc = total_char_acc / len(val_drop_last)

            scheduler.step(avg_val_loss)

            wandb.log({
                'train_loss': avg_train_loss,
                'train_accuracy': avg_train_acc,
                'train_char_accuracy': avg_train_char_acc,
                'val_loss': avg_val_loss,
                'val_accuracy': avg_val_acc,
                'val_cer': avg_val_cer,
                'val_char_accuracy': avg_val_char_acc,
                'epoch': epoch + 1
            })
            print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.3f} Acc: {avg_train_acc:.3f} | "
                  f"Val Loss: {avg_val_loss:.3f} Acc: {avg_val_acc:.3f} CER: {avg_val_cer:.3f}")

            # Early Stopping Check
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                wait = 0
                torch.save(model.state_dict(), 'best_model.pt')
            else:
                wait += 1
                if wait >= patience:
                    print("Early stopping triggered.")
                    break

        # After training, load the best model and evaluate on the test set.
        artifact = wandb.Artifact('best_model', type='model')
        artifact.add_file('best_model.pt')
        wandb.log_artifact(artifact)
        model.load_state_dict(torch.load('best_model.pt'))
        model.eval()
        test_loss = 0
        test_acc  = 0
        test_cer  = 0
        total_char_acc = 0
        with torch.no_grad():
            for source, target in test_drop_last:
                source, target = source.to(device), target.to(device)
                output = model(source, target, teacher_forcing_ratio=0)
                out_dimensions = output.shape[-1]
                loss = criterion(output[:, 1:].reshape(-1, out_dimensions), target[:, 1:].reshape(-1))
                eos_idx = target_vocab['<eos>']

                raw_preds = output.argmax(2)[:, 1:].tolist()
                raw_targets = target[:, 1:].tolist()
                preds = [strip_after_eos(p, eos_idx) for p in raw_preds]
                targets = [strip_after_eos(t, eos_idx) for t in raw_targets]
                acc = calculate_word_accuracy(preds, targets, pad_idx=target_vocab['<pad>'])
                char_acc = calculate_accuracy(preds, targets, pad_idx=target_vocab['<pad>'])
                test_loss += loss.item()
                test_acc  += acc
                preds = output.argmax(2).tolist()
                targets = target.tolist()
                test_cer += calculate_cer(preds, targets, pad_idx=target_vocab['<pad>'])
                total_char_acc += char_acc
        avg_test_loss = test_loss / len(test_drop_last)
        avg_test_acc  = test_acc / len(test_drop_last)
        avg_test_cer  = test_cer / len(test_drop_last)
        avg_test_char_acc = total_char_acc / len(test_drop_last)
        print(f"Test Loss: {avg_test_loss:.3f} | Test Acc: {avg_test_acc:.3f} | Test CER: {avg_test_cer:.3f}, Test Char Acc: {avg_test_char_acc:.3f}")

sweep_config = {
    'method': 'grid',
    'parameters': {
        'embed_dim': {'values': [256]},
        'hidden_dim_config': {'values': ['double']},
        'layers': {'values': [2]},
        'dropout': {'values': [0.3]},
        'lr': {'values': [0.001]},
        'cell_type': {'values': ['gru','lstm']},
        'teacher_forcing': {'values': [0.5]},
        'batch_size': {'value': 64},
        'epochs': {'value': 20}
    }
}

cuda


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sai-sakunthala (sai-sakunthala-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
try:
    sweep_id = wandb.sweep(sweep_config, project="dakshina-seq2seq_2")
    wandb.agent(sweep_id, function=run, count=2)
except:
    wandb.finish()

Create sweep with ID: 6cyfk5ca
Sweep URL: https://wandb.ai/sai-sakunthala-indian-institute-of-technology-madras/dakshina-seq2seq_2/sweeps/6cyfk5ca


wandb: Agent Starting Run: 2vgrjsm8 with config:
wandb: 	batch_size: 64
wandb: 	cell_type: gru
wandb: 	dropout: 0.3
wandb: 	embed_dim: 256
wandb: 	epochs: 20
wandb: 	hidden_dim_config: double
wandb: 	layers: 2
wandb: 	lr: 0.001
wandb: 	teacher_forcing: 0.5


Epoch 1 | Train Loss: 1.077 Acc: 0.113 | Val Loss: 1.174 Acc: 0.233 CER: 1.336
Epoch 2 | Train Loss: 0.612 Acc: 0.217 | Val Loss: 1.093 Acc: 0.270 CER: 1.305
Epoch 3 | Train Loss: 0.555 Acc: 0.240 | Val Loss: 1.045 Acc: 0.270 CER: 1.299
Epoch 4 | Train Loss: 0.526 Acc: 0.250 | Val Loss: 1.084 Acc: 0.275 CER: 1.297
Epoch 5 | Train Loss: 0.510 Acc: 0.255 | Val Loss: 1.053 Acc: 0.287 CER: 1.290
Epoch 6 | Train Loss: 0.507 Acc: 0.257 | Val Loss: 1.088 Acc: 0.287 CER: 1.301
Epoch 7 | Train Loss: 0.446 Acc: 0.288 | Val Loss: 1.032 Acc: 0.301 CER: 1.284
Epoch 8 | Train Loss: 0.421 Acc: 0.296 | Val Loss: 1.015 Acc: 0.310 CER: 1.281
Epoch 9 | Train Loss: 0.411 Acc: 0.300 | Val Loss: 1.057 Acc: 0.314 CER: 1.285
Epoch 10 | Train Loss: 0.404 Acc: 0.302 | Val Loss: 1.019 Acc: 0.317 CER: 1.279
Epoch 11 | Train Loss: 0.402 Acc: 0.304 | Val Loss: 1.016 Acc: 0.313 CER: 1.277
Epoch 12 | Train Loss: 0.370 Acc: 0.322 | Val Loss: 1.041 Acc: 0.318 CER: 1.275
Epoch 13 | Train Loss: 0.364 Acc: 0.324 | Val Los

Traceback (most recent call last):
  File "<ipython-input-5-52f442605f6d>", line 228, in run
    loss.backward()
  File "/usr/local/lib/python3.11/dist-packages/torch/_tensor.py", line 626, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.11/dist-packages/torch/autograd/__init__.py", line 347, in backward
    _engine_run_backward(
  File "/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py", line 823, in _engine_run_backward
    return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Exception


epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_accuracy,▁▄▅▅▆▆▇▇▇▇▇█████
train_char_accuracy,▁▆▆▆▇▇▇▇▇███████
train_loss,█▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▄▄▄▅▅▆▇████████
val_cer,█▄▄▄▃▄▂▂▂▂▁▁▁▂▁▁
val_char_accuracy,▁▄▅▅▆▆▇▇▇▇███▇██
val_loss,█▅▃▄▃▄▂▁▃▂▁▂▁▃▂▂
epoch,16
train_accuracy,0.33224
train_char_accuracy,0.86395


wandb: Agent Starting Run: z1mdfa4x with config:
wandb: 	batch_size: 64
wandb: 	cell_type: lstm
wandb: 	dropout: 0.3
wandb: 	embed_dim: 256
wandb: 	epochs: 20
wandb: 	hidden_dim_config: double
wandb: 	layers: 2
wandb: 	lr: 0.001
wandb: 	teacher_forcing: 0.5


Epoch 1 | Train Loss: 1.516 Acc: 0.057 | Val Loss: 1.100 Acc: 0.204 CER: 1.345
Epoch 2 | Train Loss: 0.633 Acc: 0.213 | Val Loss: 1.061 Acc: 0.260 CER: 1.298
Epoch 3 | Train Loss: 0.540 Acc: 0.250 | Val Loss: 1.002 Acc: 0.291 CER: 1.281
Epoch 4 | Train Loss: 0.490 Acc: 0.267 | Val Loss: 1.057 Acc: 0.306 CER: 1.287
Epoch 5 | Train Loss: 0.461 Acc: 0.279 | Val Loss: 1.025 Acc: 0.303 CER: 1.281
Epoch 6 | Train Loss: 0.440 Acc: 0.287 | Val Loss: 0.983 Acc: 0.305 CER: 1.281
Epoch 7 | Train Loss: 0.424 Acc: 0.290 | Val Loss: 1.035 Acc: 0.308 CER: 1.278
Epoch 8 | Train Loss: 0.411 Acc: 0.298 | Val Loss: 0.996 Acc: 0.315 CER: 1.276
Epoch 9 | Train Loss: 0.405 Acc: 0.301 | Val Loss: 0.971 Acc: 0.320 CER: 1.276
Epoch 10 | Train Loss: 0.402 Acc: 0.302 | Val Loss: 1.028 Acc: 0.318 CER: 1.278
Epoch 11 | Train Loss: 0.385 Acc: 0.308 | Val Loss: 1.003 Acc: 0.319 CER: 1.275
Epoch 12 | Train Loss: 0.380 Acc: 0.308 | Val Loss: 1.061 Acc: 0.319 CER: 1.281


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
wandb.finish()

epoch,▁▃▁▅▆▃█
train_accuracy,▃▇▁▇█▆█
train_char_accuracy,▄▇▁██▇█
train_loss,▅▂█▁▁▂▁
val_accuracy,▅▇▁▇▇▇█
val_cer,▄▂█▃▁▂▁
val_char_accuracy,▄▆▁▆▇▆█
val_loss,█▆▇▂▄▅▁
epoch,5
train_accuracy,0.26275
train_char_accuracy,0.82228
